<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/PS_s4e10_No_Keras%2C_No_Loan_(cv_0_963)_%F0%9F%A6%88%F0%9F%A6%88%F0%9F%A6%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s4e10_path = kagglehub.competition_download('playground-series-s4e10')
chilledwanker_loan_approval_prediction_path = kagglehub.dataset_download('chilledwanker/loan-approval-prediction')

print('Data source import complete.')


# Intro

Here I'm fitting a NN model binary_crossentropy loss.

We can get funkier with tuning, architecture and data prep.

I chose use a NN because of how successful catboost is when treating all variables as categorical.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
from pathlib import Path

from plotnine import *
import os
# suppress tf informational and warning messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras import backend as K

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import preprocessing
from sklearn import metrics

import category_encoders as ce


%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_STATE = 1966
keras.utils.set_random_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# DATA


In [ ]:
INPUT_DIR = Path('/kaggle/input/playground-series-s4e10/')

TARGET_NAMES = ['loan_status']
TARGET_NAME = 'loan_status'

test_data = pd.read_csv(INPUT_DIR / 'test.csv').assign(source=0)
train_data = pd.read_csv(INPUT_DIR / 'train.csv').assign(source=0)
original_data = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv').assign(source=1)

original_data['id'] = np.arange(len(original_data)) - 1E6
# matched with train
original_data = original_data.query('id not in [-999794, -993298]')

def to_rank(col):
    # descritise from 0..N
    return col.fillna(-1).rank(method='dense').astype('int') - 1

def fe(df):
    cat_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
    # treat continuous as categorical ranks:
    df['cb_person_cred_hist_length'] = to_rank(df['cb_person_cred_hist_length'])
    df['loan_amnt'] = to_rank(df['loan_amnt'])
    df['person_income'] = to_rank(df['person_income'])
    df['loan_int_rate'] = to_rank(df['loan_int_rate'])
    df['person_emp_length'] = to_rank(df['person_emp_length'])
    df['loan_percent_income'] = to_rank(df['loan_percent_income'])
    df['person_age'] = to_rank(df['person_age'])
    for col in cat_cols:
        # count + rank encoding, less to more frequent:
        col_series = df[col].fillna('#NA#')
        mapping = col_series.value_counts().to_dict()
        code_as = 0
        for i, key in enumerate(reversed(mapping)):
            mapping[key] = code_as
            code_as += 1
        df[col] = col_series.map(mapping)
        df[col] = df[col].astype('int')
    return df


df_all = fe(pd.concat([train_data, test_data, original_data]))

idxs = (~df_all[TARGET_NAMES[0]].isna()) & (df_all['source'] == 0)
train_data = df_all[idxs].reset_index(drop=True)
idxs = ( df_all[TARGET_NAMES[0]].isna()) & (df_all['source'] == 0)
test_data = df_all[idxs].drop(columns=[TARGET_NAMES[0]])
original_data = df_all.query('source == 1')

cont_features = ['cb_person_default_on_file', 'source']
cat_features = [
    'person_home_ownership',
    'loan_intent',
    'loan_grade',
    'person_emp_length',
    'loan_int_rate',
    'loan_percent_income',
    'person_age',
    'person_income',
    'loan_amnt',
    'cb_person_cred_hist_length']

cat_features_card = {}
for f in cat_features:
    cat_features_card[f] = 1 + df_all[f].max()

df_all = None

features = cat_features + cont_features

In [ ]:
train_data.filter(features).describe(include='all').T

# Model Architecture

Less is more.

Maybe a little more might be better here :)

In [ ]:
def build_model(cat_features, cont_features):

    # Define input layers
    cat_inputs = [layers.Input(shape=(1,), name=f'cat{i}') for i in range(len(cat_features))]
    cont_inputs = layers.Input(shape=(len(cont_features),))

    # Embedding layers for categorical inputs
    flat_embeddings = []
    for i, f in enumerate(cat_features):
        input_dim = int(cat_features_card[f])
        output_dim = int(min(128, round(1.6 * input_dim ** .56))) # based on the fastai library

        embedding = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)(cat_inputs[i])
        if output_dim > 32:
            embedding = layers.SpatialDropout1D(.5)(embedding)
        else:
            embedding = layers.SpatialDropout1D(.3)(embedding)
        flat_embeddings.append(layers.Flatten()(embedding))

    concatenated_inputs = layers.Concatenate()(flat_embeddings + [cont_inputs, ])
    concatenated_inputs_bn = layers.BatchNormalization()(concatenated_inputs)

    x = layers.Dense(256, activation='mish')(concatenated_inputs_bn)

    for units in (128,):
        inp = layers.Concatenate()([x, concatenated_inputs_bn])
        x = layers.Dense(units=units, activation='mish')(inp)
        x = layers.Dropout(.3)(x)

    # output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return keras.Model(cat_inputs + [cont_inputs], outputs)

# initially use a single learning rate
# for a small number of epochs
epochs = 6
callbacks = []

In [ ]:
tf.keras.utils.plot_model(
    model=build_model(cat_features, cont_features),
    show_shapes=True,
    rankdir='TB')

# Fit

In [ ]:
def fold_logloss(y, preds):
    return metrics.log_loss(y, preds)

def fold_auc(y, preds):
    return metrics.roc_auc_score(y, preds)

# to feed data into the NN
# we feed the categoricals column by column,
# and the continuous features in one lump.
cat_idxs= []
cont_idxs = []
for f in cat_features:
    cat_idxs.append([features.index(f)])
for f in cont_features:
    cont_idxs.append(features.index(f))

feature_idxs = cat_idxs + [cont_idxs]

def to_nn_feed(df):
    X = df[feats].values
    result = []
    for f_idx in feature_idxs:
        # housekeeping: to feed data into the NN
        # we feed the categoricals column by column,
        # and the continuous features in one lump.
        result.append(X[:, f_idx])
    return result

def fit_fold(tr, vl, ts):

    model = build_model(cat_features, cont_features)
    model.compile(
        optimizer=keras.optimizers.AdamW(learning_rate=3E-4),
        loss='binary_crossentropy',
        metrics=['auc'])

    history = model.fit(
          to_nn_feed(tr), tr[TARGET_NAME],
          #validation_data=(to_nn_feed(vl), vl[TARGET_NAME]),
          batch_size=BS,
          epochs=epochs,
          callbacks=callbacks,
          verbose=0
    )

    vl_pred = model.predict(to_nn_feed(vl), verbose=0, batch_size=BS).flatten()
    ts_pred = model.predict(to_nn_feed(ts), verbose=0, batch_size=BS).flatten()

    vl_metric = fold_auc(vl[TARGET_NAME], vl_pred)
    return vl_pred, ts_pred, vl_metric

In [ ]:
N_FOLDS = 5
BS = 256
feats = features

vl_preds = np.zeros(len(train_data))
ts_preds = np.zeros(len(test_data))
vl_metrics = []

keras.utils.set_random_seed(RANDOM_STATE)
k_fold = StratifiedKFold(n_splits=N_FOLDS, random_state=RANDOM_STATE, shuffle=True)
for tr_idx, vl_idx in k_fold.split(train_data, train_data[TARGET_NAME]):
    tr = train_data.loc[tr_idx]
    vl = train_data.loc[vl_idx]

    # add original data to the training fold only:
    vl_pred, ts_pred, vl_metric = fit_fold(pd.concat([tr, original_data]), vl, test_data)

    print(f'  -- fold auc {vl_metric:2.4f}')
    vl_metrics.append(vl_metric)
    vl_preds[vl_idx] += vl_pred
    ts_preds += ts_pred / N_FOLDS

# overall metric:
vl_metric = fold_auc(
    train_data[TARGET_NAME], vl_preds)
print(f'  ----------- {vl_metric:2.4f}')

# How we doin'?

In [ ]:
df_metrics = pd.DataFrame({'oof': vl_metrics})
print(ggplot(df_metrics, aes(x='oof'))
    + theme_light()
    + theme(figure_size=(6, 3))
    + geom_dotplot(alpha=0.3, binwidth = 1E-4, fill='SteelBlue')
    + geom_vline(xintercept=np.mean(vl_metrics), linetype='dashed')
    + labs(y = '', x='', title = f'OOF Metric Distribution\nAUC={np.mean(vl_metrics):0.4f}'))

# Submission

In [ ]:
# oof data
(train_data
    .filter(['id', TARGET_NAME])
    .assign(oof = vl_preds)
    .to_csv('oof.csv', index=False))

pd.read_csv('oof.csv').head(20)

In [ ]:
submission_filename = f'submission_{np.mean(vl_metrics):0.4f}.csv'

(test_data
    .assign(**{
        TARGET_NAME: ts_preds})
    .to_csv(submission_filename, columns=['id', TARGET_NAME], index=False))

In [ ]:
pd.read_csv(submission_filename).head(20)